In [0]:
spark.sql("""
EXPLAIN EXTENDED
SELECT *
FROM ecommerce.silver.events
WHERE event_type = 'purchase'
""")


DataFrame[plan: string]

In [0]:
%sql
CREATE OR REPLACE TABLE ecommerce.silver.events_part
USING DELTA
PARTITIONED BY (event_type)
AS
SELECT *
FROM ecommerce.silver.events;


num_affected_rows,num_inserted_rows


In [0]:
%sql
OPTIMIZE ecommerce.silver.events_part
ZORDER BY (user_id, product_id);


path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(0, 0), 0, List(0, 0), 0, null), null, 0, 0, 0, 0, false, 0, 0, 1768719905077, 1768719905996, 8, 0, null, List(0, 0), null, 9, 9, 0, 0, null)"


In [0]:
import time

start = time.time()
spark.sql("""
SELECT COUNT(*)
FROM ecommerce.silver.events
WHERE user_id = 12345
""").collect()

print(f"Before optimization: {time.time() - start:.2f} seconds")


Before optimization: 0.57 seconds


In [0]:
start = time.time()
spark.sql("""
SELECT COUNT(*)
FROM ecommerce.silver.events_part
WHERE user_id = 12345
""").collect()

print(f"After optimization: {time.time() - start:.2f} seconds")


After optimization: 0.44 seconds


In [0]:
optimized_df = spark.table("ecommerce.silver.events_part")
optimized_df.createOrReplaceTempView("events_cached_view")


In [0]:
%sql
SELECT COUNT(*)
FROM events_cached_view
WHERE user_id = 12345;


COUNT(*)
0
